In [ ]:
#############

In [ ]:
##################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F


In [ ]:

# Define the DeepFool attack function
def deepfool(image, net, num_classes=10, overshoot=0.02, max_iter=50):
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    net.eval()  # Set model to evaluation mode
    f_image = net.forward(image).data.cpu().numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]
    I = I[0:num_classes]
    label = I[0]
    input_shape = image.cpu().detach().numpy().shape
    pert_image = image.cpu().detach().numpy()
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0
    while label == I[0] and loop_i < max_iter:

        pert = np.inf
        gradients = np.zeros(input_shape)

        for k in range(1, num_classes):
            x = np.copy(w)
            w_norm = np.linalg.norm(w.flatten())
            if w_norm == 0:
                r_i = (overshoot / np.sqrt(np.prod(input_shape))) * np.sign(np.random.randn(*input_shape))
            else:
                r_i = (overshoot / w_norm) * w
            pert_image = image + (1 + overshoot) * torch.from_numpy(r_i).float().to(device)
            pert_image.requires_grad = True
            net.zero_grad()
            x = torch.from_numpy(x).float().to(device)
            output = net.forward(pert_image)
            loss = nn.CrossEntropyLoss()(output, torch.tensor([label]).to(device))
            loss.backward()
            grad = pert_image.grad.cpu().numpy()
            gradients = gradients + grad

            if k == 1:
                w = np.copy(gradients)
            else:
                w = np.copy(gradients) - np.sum(gradients * w, axis=3, keepdims=True) * w / np.linalg.norm(w.flatten())

            w_norm = np.linalg.norm(w.flatten())
            if w_norm == 0:
                r_i = (overshoot / np.sqrt(np.prod(input_shape))) * np.sign(np.random.randn(*input_shape))
            else:
                r_i = (overshoot / w_norm) * w

            pert = np.linalg.norm(r_i)

            if pert < 0.1:
                r_tot = r_tot + r_i
                break

        loop_i += 1

        pert_image = image + torch.from_numpy(r_tot).float().to(device)

        f_pert = net.forward(pert_image).data.cpu().numpy().flatten()
        if (np.array(f_pert)).flatten().argsort()[::-1][0] != label:
            r_tot = np.zeros(input_shape)

    return torch.from_numpy(r_tot).float().to(device)

In [ ]:
# Load the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:

# Define the training function
def train(model, optimizer, criterion, trainloader, device, num_epochs):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Apply DeepFool attack to the inputs
            perturbations = deepfool(inputs, model)
            perturbed_inputs = inputs + perturbations

            outputs = model(perturbed_inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / (i + 1)))


In [ ]:
# Train the neural network using federated learning
num_clients = 10
local_epochs = 10
num_epochs = 50
lr = 0.01

In [ ]:
# Split the CIFAR-10 dataset into non-iid portions for each client
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_partitions = {}
label_partitions = {}
for i in range(num_clients):
    data_partitions[i] = trainset.data[i::num_clients]
    label_partitions[i] = trainset.targets[i::num_clients]

clients = {}
for i in range(num_clients):
    clients[i] = {}
    clients[i]['data'] = torch.from_numpy(data_partitions[i]).to(device)
    clients[i]['targets'] = torch.tensor(label_partitions[i], dtype=torch.long).to(device)
    clients[i]['model'] = Net().to(device)
    clients[i]['optimizer'] = optim.SGD(clients[i]['model'].parameters(), lr=lr)
    clients[i]['criterion'] = nn.CrossEntropyLoss()

global_model = Net().to(device)

for epoch in range(num_epochs):
    print('Global epoch %d' % (epoch + 1))
    global_model.train()

    # Train local models
    for i in range(num_clients):
        print('Client %d training' % i)
        train(clients[i]['model'], clients[i]['optimizer'], clients[i]['criterion'], \
              torch.utils.data.TensorDataset(clients[i]['data'], clients[i]['targets']), device, local_epochs)

        # Update the global model with the local model weights
        for global_param, local_param in zip(global_model.parameters(), clients[i]['model'].parameters()):
            global_param.data += local_param.data

Global epoch 1
Client 0 training


RuntimeError: ignored

In [ ]:
# Average the global model weights over all clients
for global_param in global_model.parameters():
    global_param.data /= num_clients

In [ ]:
# Evaluate the global model on the test set
global_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = global_model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: %d %%' % (100 * correct / total))

In [ ]:
###################

In [ ]:
#################

In [ ]:
##################

In [ ]:
#################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import copy
from torchvision import models

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set up the transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def deepfool_attack(model, x, num_classes=10, max_iter=50, epsilon=0.02):
    # Convert x to a tensor
    x = x.clone().detach().requires_grad_(True).to(device)
    # Get the output of the model
    output = model(x.unsqueeze(0))
    # Get the predicted label
    label = torch.argmax(output, dim=1)
    # Initialize the perturbation
    perturbation = torch.zeros_like(x)
    # Loop until the perturbation fools the model
    for i in range(max_iter):
        # Calculate the gradients of the output with respect to x
        output[0, label].backward(retain_graph=True)
        grads = x.grad.clone().detach()
        # Calculate the perturbation
        w = torch.zeros((num_classes, 1)).to(device)
        f = torch.zeros((num_classes, 1)).to(device)
        for j in range(num_classes):
            if j == label:
                continue
            output[0, j].backward(retain_graph=True)
            w[j] = x.grad.clone().detach().view(-1)
            f[j] = output[0, j].clone().detach() - output[0, label].clone().detach()
            x.grad.data.zero_()
        f_prime = torch.abs(f)
        index_sorted = torch.argsort(f_prime, dim=0, descending=True)
        t = 0
        sum_pert = torch.zeros_like(x)
        while t < num_classes - 1 and torch.argmax(f_prime[index_sorted[t]]) == label:
            j = index_sorted[t]
            pert = (abs(f[j]) + epsilon) / torch.norm(w[j])
            sum_pert += pert * w[j].view(x.shape)
            t += 1
        perturbation += sum_pert
        perturbed_x = x + perturbation 
        # Check if the perturbed image fools the model
        perturbed_output = model(perturbed_x.unsqueeze(0))
        perturbed_label = torch.argmax(perturbed_output, dim=1)
        if perturbed_label != label:
            break
        else:
            perturbation *= 0.9
    return perturbed_x.detach()


In [ ]:
# # Define the CNN architecture
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
#         self.relu1 = nn.ReLU()
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
#         self.relu3 = nn.ReLU()
#         self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(4*4*128, 256)
#         self.relu4 = nn.ReLU()
#         self.dropout1 = nn.Dropout(p=0.5)
#         self.fc2 = nn.Linear(256, 10)

#     def forward(self, x):
#         out = self.conv1(x)
#         out = self.relu1(out)
#         out = self.maxpool1(out)
#         out = self.conv2(out)
#         out = self.relu2(out)
#         out = self.maxpool2(out)
#         out = self.conv3(out)
#         out = self.relu3(out)
#         out = self.maxpool3(out)
#         out = out.view(out.size(0), -1)
#         out = self.fc1(out)
#         out = self.relu4(out)
#         out = self.dropout1(out)
#         out = self.fc2(out)
#         return out


class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

# model = ResNet18().to(device)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = transforms.Resize(224)(data)
        data, target = data.to(device), target.to(device)

        # Generate adversarial examples using the DeepFool attack for each image in the batch
        perturbed_batch = []
        for img_idx in range(data.shape[0]):
            adv_example = deepfool_attack(model, data[img_idx])
            perturbed_batch.append(adv_example.unsqueeze(0))
        perturbed_batch = torch.cat(perturbed_batch, dim=0)

        # Pass the perturbed batch through the model
        output = model(perturbed_batch)

        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_acc = 100. * correct / total
    train_loss /= len(train_loader)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, total, train_acc))
    return train_loss, train_acc


In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
    test_acc = 100. * correct / total
    test_loss /= len(test_loader)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, total, test_acc))
    return test_loss, test_acc

In [ ]:
# Define the Federated Learning function
def federated_learning(model, train_loaders, test_loader, num_rounds=10, lr=0.1, momentum=0.9, epsilon=0.03):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    test_loss = []
    test_acc = []
    for round_ in range(num_rounds):
        # Clone the model for each client
        client_models = [copy.deepcopy(model) for _ in range(len(train_loaders))]
        # Train each client model
        train_losses = []
        train_accs = []
        for i, (train_loader, client_model) in enumerate(zip(train_loaders, client_models)):
            print(f"Round: {round_}, Client: {i+1}")
            client_model.train()
            optimizer = optim.SGD(client_model.parameters(), lr=lr, momentum=momentum)
            train_loss, train_acc = train(client_model, train_loader, optimizer, criterion, round_)
            train_losses.append(train_loss)
            train_accs.append(train_acc)
        # Aggregate the client models using Federated Averaging
        for param in model.parameters():
            param.data = torch.zeros_like(param.data)
        for client_model in client_models:
            for param, client_param in zip(model.parameters(), client_model.parameters()):
                param.data += client_param.data / len(client_models)
        
        # Evaluate the aggregated model
        test_loss_round, test_acc_round = test(model, test_loader, criterion)
        test_loss.append(test_loss_round)
        test_acc.append(test_acc_round)

    return train_losses, train_accs, test_loss, test_acc


In [ ]:
# Set up the data and model
num_clients = 10
epsilon=0.03
momentum = 0.9
lr=0.01
train_loaders = [torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True) for _ in range(num_clients)]
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False) 
model = ResNet18().to(device)

# Train the model
train_losses, train_accs, test_loss, test_acc = federated_learning(model, train_loaders, test_loader, lr=lr, momentum=momentum, epsilon=epsilon)

# Print the results
print('Train Losses:', train_losses)
print('Train Accuracies:', train_accs)
print('Test Losses:', test_loss)
print('Test Accuracies:', test_acc)


Round: 0, Client: 1


RuntimeError: ignored

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F


# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:

# # Define deepfool attack function
# def deepfool_attack(image, model, num_classes=10, overshoot=0.02, max_iter=50):
#     """
#     Deepfool attack implementation for CIFAR-10 images.
#     """
#     # Convert image to tensor and send to device
#     image = image.to(device)
#     image.requires_grad = True
    
#     # Get predicted label for image
#     output = model(image)
#     init_pred = output.max(1, keepdim=True)[1].item()
    
#     # Initialize perturbation
#     perturbation = torch.zeros_like(image).to(device)
#     perturbed_image = torch.zeros_like(image).to(device)
    
#     # Initialize counter and loop until the image is misclassified
#     i = 0
#     while init_pred == output.max(1, keepdim=True)[1].item() and i < max_iter:
#         # Calculate gradients and get predicted label
#         output.backward()
#         grad = image.grad.data
#         pred_label = output.max(1, keepdim=True)[1].item()
        
#         # Initialize min and min_index
#         min_norm = float('inf')
#         min_index = None
        
#         # Loop through all classes
#         for k in range(num_classes):
#             # Skip if class is equal to predicted class
#             if k == pred_label:
#                 continue
            
#             # Calculate perturbation for class k
#             w_k = grad[k, :, :, :].clone()
#             w = grad[pred_label, :, :, :].clone()
#             f_k = output[0, k].item()
#             f = output[0, pred_label].item()
#             pert_k = abs(f_k - f) / torch.norm(w_k - w)
            
#             # Update min_norm and min_index if perturbation for class k is smaller
#             if pert_k < min_norm:
#                 min_norm = pert_k
#                 min_index = k
        
#         # Update perturbation and perturbed_image
#         r = min_norm * (w - w_k)
#         perturbation += r
#         perturbed_image = image + (1 + overshoot) * perturbation
        
#         # Clamp the perturbed image to valid range
#         perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()
        
#         # Reset gradients and increment counter
#         model.zero_grad()
#         output = model(perturbed_image)
#         i += 1
        
#     return perturbed_image.detach(), init_pred, output.max(1, keepdim=True)[1].item()



def deepfool_attack(image, model, num_classes=10, overshoot=0.02, max_iter=50):
    """
    Deepfool attack implementation for CIFAR-10 images.
    """
    # Convert image to tensor and send to device
    image = image.to(device)
    image.requires_grad = True
    
    # Get predicted label for image
    with torch.enable_grad():
        output = model(image)
    # init_pred = output.max(1, keepdim=True)[1].item()
    
    # Check if output tensor is empty
    if output.numel() == 0:
        return None, None, None
    output_sum = output.sum()
    init_pred = output.max(1, keepdim=True)[1].item()
    # Initialize perturbation
    perturbation = torch.zeros_like(image).to(device)
    perturbed_image = torch.zeros_like(image).to(device)
    
    # Initialize counter and loop until the image is misclassified
    i = 0
    while init_pred == output.max(1, keepdim=True)[1].item() and i < max_iter and output.numel() > 0:

        # Calculate gradients and get predicted label
        image.grad = torch.zeros_like(image)
        grad = image.grad.data
        
        output.backward(torch.ones_like(output), retain_graph=True)
        grad = image.grad.clone().detach()

        pred_label = output.max(1, keepdim=True)[1].item()
        
        # Initialize min and min_index
        min_norm = float('inf')
        min_index = None
        
        # Loop through all classes
        for k in range(num_classes):
            # Skip if class is equal to predicted class
            if k == pred_label:
                continue
            
            # Calculate perturbation for class k
            w_k = grad.clone()
            w_k[0, pred_label] = 0
            w = grad.clone()
            w[0, k] = 0


            f_k = output[0, k].item()
            f = output[0, pred_label].item()
            pert_k = abs(f_k - f) / torch.norm(w_k - w)
            
            # Update min_norm and min_index if perturbation for class k is smaller
            if pert_k < min_norm:
                min_norm = pert_k
                min_index = k
        
        # Update perturbation and perturbed_image
        r = min_norm * (w - w_k)
        perturbation += r
        perturbed_image = image + (1 + overshoot) * perturbation

        # Clamp the perturbed image to valid range
        perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()


        
        # # Update perturbation and perturbed_image
        # r = min_norm * (w - w_k)
        # perturbation += r
        # perturbed_image = image + (1 + overshoot) * perturbation
        
        # # Clamp the perturbed image to valid range
       


        
        # # Update perturbation and perturbed_image
        # r = min_norm * (w - w_k)
        # perturbation += r
        # perturbed_image = image + (1 + overshoot) * perturbation
        
        # # Clamp the perturbed image to valid range
        # perturbed_image = torch.clamp(perturbed_image, 0, 1).detach()
        
        # Reset gradients and increment counter
        model.zero_grad()
        with torch.enable_grad():
            output = model(perturbed_image)
            output_sum = output.sum()
        i += 1



        
    return perturbed_image.detach(), init_pred, output.max(1, keepdim=True)[1].item()




In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * 128, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 4 * 4 * 128)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


# Initialize model and optimizer
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
log_interval=10
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if data.shape[0] == 1:
            continue
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    return model





In [7]:
# def test(model, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     accuracy = 100. * correct / len(test_loader.dataset)
#     print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")
#     return accuracy


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Apply deepfool attack on first image in batch
            perturbed_image, init_pred, adv_pred = deepfool_attack(data[0], model)

            # Get predicted label for perturbed image
            output = model(perturbed_image.unsqueeze(0))
            adv_pred = output.max(1, keepdim=True)[1].item()

            # Calculate loss and accuracy
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%), Adv Accuracy: {len(test_loader.dataset) - correct}/{len(test_loader.dataset)} '
          f'({100 - accuracy:.2f}%)')
    return accuracy


In [8]:
# Initialize Federated Learning setup
# Initialize Federated Learning setup
# Initialize Federated Learning setup
num_clients = 10
train_loaders = []
test_loaders = []



trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_subsplits = torch.utils.data.random_split(trainset, [int(len(train_dataset)/num_clients) for _ in range(num_clients)])
test_subsplits = torch.utils.data.random_split(testset, [int(len(test_dataset)/num_clients) for _ in range(num_clients)])

for i in range(num_clients):
    train_loader = torch.utils.data.DataLoader(train_subsplits[i], batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_subsplits[i], batch_size=128, shuffle=False)
    
    train_loaders.append(train_loader)
    test_loaders.append(test_loader)



# Define Federated Learning function
def federated_learning(model, train_loaders, test_loaders, optimizer, num_epochs):
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}\n-------------------------------")
        for i in range(num_clients):
            print(f"Client {i + 1}")
            model = train(model, train_loaders[i], optimizer, epoch + 1)
        test_accs = []
        for i in range(num_clients):
            test_acc = test(model, test_loaders[i])
            test_accs.append(test_acc)
        print(f"Test Accuracy: {sum(test_accs) / len(test_accs)}")
    
    return model

# Train the model using Federated Learning
model = federated_learning(model, train_loaders, test_loaders, optimizer, num_epochs=5)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1
-------------------------------
Client 1
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.302668
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.290805
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.262135
Train Epoch: 1 [3840/5000 (75%)]	Loss: 2.184063
Client 2
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.237004
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.215704
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.183702
Train Epoch: 1 [3840/5000 (75%)]	Loss: 2.154138
Client 3
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.184165
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.171565
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.249456
Train Epoch: 1 [3840/5000 (75%)]	Loss: 2.186972
Client 4
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.135293
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.180833
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.171788
Train Epoch: 1 [3840/5000 (75%)]	Loss: 2.162971
Client 5
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.152950
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.14

KeyboardInterrupt: ignored